In [1]:
import os
import shutil

In [2]:
from ultralytics import YOLO

In [ ]:
'''
Diversas vezes nas células, os modelos serão referidos como 'modelo funcional' e 'experimentos':

- Modelo Funcional = modelo que teve o melhor desempenho na detecção de objetos cortantes do vídeo disponibilizado, escolhido para avaliação.
- Experimentos = Diversos treinamentos experimentais feitos, combinando vários hiperparâmetros e versões distintas do YOLO. São os que não tiveram um desempenho
                 melhor em comparação ao modelo funcional, mas que tiveram ou não um melhor desempenho nas métricas e estatísticas do ultralytics e até mesmo
                 nas imagens de 'test' quando postas sob avaliação.
'''

In [3]:
class ModeloYolo:
    def __init__(self, modelo_path, output_dir):
        self.modelo = YOLO(modelo_path)
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)

    def treinar(self, dataset_path, epochs_treinamento, imgsz, batch_size, lr0, lrf, augment, verbose, resume, device):
        print("Iniciando o treinamento...")

        self.modelo.train(
            data=dataset_path,
            epochs=epochs_treinamento, #50
            imgsz=imgsz, #640
            batch=batch_size, #32
            lr0=lr0, #0.003
            lrf=lrf, #0.2
            augment=augment, #True
            verbose=verbose, #False
            resume=resume, #False
            device=device #0
        )
        #Hiperparâmetros extras para otimização de treinamento (não foram usados no treinamento do modelo funcional, apenas em outros experimentos):
        '''
        workers=8,
        pretrained=True,
        resume=False,
        single_cls=False, 
        box=7.5,
        cls=0.5, 
        dfl=1.5,
        val=True,
        degrees=0.3,
        hsv_s=0.3,
        hsv_v=0.3,
        scale=0.5,
        fliplr=0.5,
        classes=[0],
        patience=10
        '''
    
        # Caminho onde o YOLO salva automaticamente o melhor modelo
        best_model_path = os.path.join("runs", "detect", "train", "weights", "best.pt") #(train1, train2...)
        
        # Caminho onde salvar o best.pt (melhor modelo do treinamento)
        final_model_path = os.path.join(self.output_dir, 'best.pt')
        
        # Copiar o best.pt para o diretório desejado
        if os.path.exists(best_model_path):
            shutil.copy(best_model_path, final_model_path)
            print(f"Melhor modelo salvo em: {final_model_path}")
        else:
            print("Erro: best.pt não encontrado. Verifique o caminho de saída do YOLO.")

In [4]:
current_dir = os.getcwd()

caminho_modelo = os.path.join(current_dir, 'modelo')  # Diretório onde o modelo treinado será salvo
caminho_yaml = os.path.join(current_dir, 'dataset.yaml')  # Caminho para o arquivo YAML de configuração do dataset

# Inicializar e treinar o modelo
modelo = os.path.join(caminho_modelo,'yolo11s.pt') #yolo11s utilizado tanto no modelo funcional quanto modelo robusto (experimento), o yolo11m não apresentou melhoras significativas

classe_modelo = ModeloYolo(modelo, caminho_modelo)

'''
Considerações antes de executar o treino:
    - Device=0 caso possua uma gpu, do contrário, adote 'cpu'
    - O treinamento do modelo funcional foi feito com uma placa de vídeoo RTX 3060 TI, tomando 4h de tempo de treinamento com os parâmetros abaixo,
      portanto, caso queira apenas testar o fluxo de funcionamento deste projeto, informe de 1 a 5 epochs para não tomar muito tempo. Porém, isso
      fará com que o treinamento não desempenhe o mesmo resultado obtido com o modelo funcional.
'''

classe_modelo.treinar(caminho_yaml, 50, 640, 32, 0.003, 0.2, True, False, False, 0)

'''
OBS: Durante o treinamento, o YOLO baixa o modelo yolo11n.pt para verificar a compatibilidade do seu ambiente com AMP. Essa verificação assegura que o treinamento realizado de forma eficiente, aproveitando os benefícios da precisão mista. Essa verificação não afeta o modelo que está sendo treinado. O treinamento ocorrerá com o modelo especificado 'yolo11s.pt', e o arquivo yolo11n.pt é utilizado apenas para a verificação de AMP (O AMP reduz o consumo de memória e acelera o treinamento de redes neurais).
'''

100%|██████████| 18.4M/18.4M [00:03<00:00, 5.60MB/s]


Iniciando o treinamento...
New https://pypi.org/project/ultralytics/8.3.74 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.69  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
engine\trainer: task=detect, mode=train, model=C:\Users\Pichau\PycharmProjects\HACKATHON-MAIN\Hackathon\modelo\yolo11s.pt, data=C:\Users\Pichau\PycharmProjects\HACKATHON-MAIN\Hackathon\dataset.yaml, epochs=3, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1,

100%|██████████| 5.35M/5.35M [00:01<00:00, 4.51MB/s]


AMP: checks passed 


train: Scanning C:\Users\Pichau\PycharmProjects\HACKATHON-MAIN\Hackathon\arquivo\imagens\dataset-hackathon\train\labels... 4409 images, 20 backgrounds, 0 corrupt: 100%|██████████| 4409/4409 [00:04<00:00, 1073.84it/s]


...

        3/3      2.36G      1.401      1.916      1.701          2        640: 100%|██████████| 552/552 [00:55<00:00,  9.99it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 66/66 [00:06<00:00,  9.74it/s]

                   all       1043       1203       0.57      0.498      0.527      0.315



3 epochs completed in 0.058 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 19.2MB
Optimizer stripped from runs\detect\train\weights\best.pt, 19.2MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.69  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
YOLO11s summary (fused): 238 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 66/66 [00:11<00:00,  5.54it/s]


                   all       1043       1203      0.688      0.499       0.59      0.391
Speed: 0.3ms preprocess, 7.4ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect\train
Melhor modelo salvo em: C:\Users\Pichau\PycharmProjects\HACKATHON-MAIN\Hackathon\modelo\best.pt


In [6]:
import glob

current_dir = os.getcwd()

caminho_imagens = os.path.join(current_dir, 'arquivo', 'imagens', 'dataset-hackathon', 'test', 'images')  # Pasta onde as imagens de teste estão
caminho_modelo = os.path.join(current_dir, 'modelo')  # Diretório do modelo
caminho_modelo_treinado = os.path.join(caminho_modelo,'best.pt')  # Diretório do modelo treinado

modelo = YOLO(caminho_modelo_treinado)
'''
Para testar com o modelo funcional:
modelo = YOLO('modelo-funcional/weights/best.pt')
'''

# Seleciona as 5 primeiras imagens da pasta de teste
imagens = sorted(glob.glob(caminho_imagens+'/*.jpg'))[:5]

modelo.predict(
    source=imagens,     # Arquivos, pasta ou URL
    imgsz=640,          # Tamanho da imagem (padrão: 640)
    conf=0.5,           # Confiança mínima para detectar um objeto
    iou=0.5,            # IOU mínimo para supressão de NMS (padrão: 0.7)
    device=0,           # Define a GPU (0) ou CPU ('cpu')
    save=True,          # Salvar imagens com predições
    save_txt=True,      # Salvar resultados em arquivos .txt
    save_conf=True,     # Incluir valores de confiança nos .txt salvos
    show=False,          # Exibir predições na tela
    line_thickness=2,   # Espessura das caixas delimitadoras
    hide_labels=False,  # Esconder os rótulos dos objetos detectados
    hide_conf=False,    # Esconder valores de confiança das caixas
    classes=[0],        # Filtrar apenas certas classes (0 = knife, 1 = pistol)
    half=True,          # Usa precisão mista (reduz consumo de memória na GPU)
    augment=False,      # Usa técnicas de aumento de dados na inferência
    agnostic_nms=True,  # Aplica NMS sem considerar classes diferentes
    visualize=True,     # Gera mapas de ativação para melhor interpretação
    project="resultado-test",   # Define a pasta de saída
    name="teste-modelo"         # Nome da subpasta dentro de 'project'
)

WARNING  'line_thickness' is deprecated and will be removed in in the future. Use 'line_width' instead.
WARNING  'hide_labels' is deprecated and will be removed in in the future. Use 'show_labels' instead.
WARNING  'hide_conf' is deprecated and will be removed in in the future. Use 'show_conf' instead.

Saving resultado-test\teste-modelo2\--------_------_jpg.rf.4eb0868f6cd41827c921043ddfa37ff9\stage0_Conv_features.png... (32/32)
Saving resultado-test\teste-modelo2\--------_------_jpg.rf.4eb0868f6cd41827c921043ddfa37ff9\stage1_Conv_features.png... (32/64)
Saving resultado-test\teste-modelo2\--------_------_jpg.rf.4eb0868f6cd41827c921043ddfa37ff9\stage2_C3k2_features.png... (32/128)
Saving resultado-test\teste-modelo2\--------_------_jpg.rf.4eb0868f6cd41827c921043ddfa37ff9\stage3_Conv_features.png... (32/128)
Saving resultado-test\teste-modelo2\--------_------_jpg.rf.4eb0868f6cd41827c921043ddfa37ff9\stage4_C3k2_features.png... (32/256)
Saving resultado-test\teste-modelo2\--------_------_

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'knife', 1: 'pistol'}
 obb: None
 orig_img: array([[[ 20,  17,  13],
         [ 21,  18,  14],
         [ 21,  18,  14],
         ...,
         [119, 162, 205],
         [120, 162, 207],
         [119, 164, 208]],
 
        [[ 18,  15,  11],
         [ 19,  16,  12],
         [ 20,  17,  13],
         ...,
         [135, 177, 220],
         [133, 175, 220],
         [130, 175, 219]],
 
        [[ 15,  12,   8],
         [ 17,  14,  10],
         [ 19,  16,  12],
         ...,
         [136, 178, 220],
         [135, 178, 221],
         [134, 177, 220]],
 
        ...,
 
        [[159, 197, 232],
         [158, 196, 231],
         [157, 195, 230],
         ...,
         [179, 182, 190],
         [148, 150, 161],
         [131, 135, 146]],
 
        [[159, 197, 232],
         [158, 196, 231],
         [157, 195, 230],
         ...,
      